```@meta
CurrentModule = BayesLaplace
```

# Bayesian MLP

In [1]:
# Import libraries.
using Flux, Plots, Random, PlotThemes, Statistics, BayesLaplace
theme(:lime)

This time we use a synthetic dataset containing samples that are not linearly separable:

In [2]:
# Number of points to generate.
xs, y = toy_data_non_linear(200)
X = hcat(xs...); # bring into tabular format
data = zip(xs,y);

For the classification task we build a neural network with weight decay composed of a single hidden layer.

In [3]:
n_hidden = 32
D = size(X)[1]
nn = Chain(
    Dense(D, n_hidden, σ),
    Dense(n_hidden, 1)
)  
λ = 0.01
sqnorm(x) = sum(abs2, x)
weight_regularization(λ=λ) = 1/2 * λ^2 * sum(sqnorm, Flux.params(nn))
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y) + weight_regularization();

The model is trained for 200 epochs before the training loss stagnates.

In [4]:
using Flux.Optimise: update!, ADAM
opt = ADAM()
epochs = 200
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))

using Plots
anim = Animation()
plt = plot(ylim=(0,avg_loss(data)), xlim=(0,epochs), legend=false, xlab="Epoch")
avg_l = []

for epoch = 1:epochs
  for d in data
    gs = gradient(params(nn)) do
      l = loss(d...)
    end
    update!(opt, params(nn), gs)
  end
  avg_l = vcat(avg_l,avg_loss(data))
  plot!(plt, avg_l, color=1, title="Average (training) loss")
  frame(anim, plt)
end

gif(anim, "www/nn_training_mlp.gif");

┌ Info: Saved animation to 
│   fn = /Users/FA31DU/Documents/code/BayesLaplace.jl/docs/src/www/nn_training_mlp.gif
└ @ Plots /Users/FA31DU/.julia/packages/Plots/8ouqB/src/animation.jl:114


![](www/nn_training_mlp.gif)

## Laplace appoximation

Laplace approximation can be implemented as follows:

In [5]:
la = laplace(nn, λ=λ, subset_of_weights=:last_layer)
fit!(la, data);
zoom=0
p_plugin = plot_contour(X',y,la;title="Plugin",type=:plugin,zoom=zoom);
p_laplace = plot_contour(X',y,la;title="Laplace",zoom=zoom);

The plot below shows the resulting posterior predictive surface for the plugin estimator (left) and the Laplace approximation (right).

In [6]:
# Plot the posterior distribution with a contour plot.
plt = plot(p_plugin, p_laplace, layout=(1,2), size=(1000,400))
savefig(plt, "www/posterior_predictive_mlp.png")

![](www/posterior_predictive_mlp.png)

Zooming out we can note that the plugin estimator produces high-confidence estimates in regions scarce of any samples. The Laplace approximation is much more conservative about these regions.

In [7]:
zoom=-50
p_plugin = plot_contour(X',y,la;title="Plugin",type=:plugin,zoom=zoom);
p_laplace = plot_contour(X',y,la;title="Laplace",zoom=zoom);
# Plot the posterior distribution with a contour plot.
plt = plot(p_plugin, p_laplace, layout=(1,2), size=(1000,400))
savefig(plt, "www/posterior_predictive_mlp_zoomed.png");

![](www/posterior_predictive_mlp_zoomed.png)